# Notebook - Battle Of Neighborhoods

This is the notebook for the Capstone Project - week2.

## I. Setup Environment

In [263]:
# Battle Of Neighborhoods - Week2

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## II. Scraping California City Information From Wikipedia

### a. City Name, Population and Size

In [264]:
# Gathering California City Info From Wikipedia Page
from bs4 import BeautifulSoup

# Get wikipedia page about California cities to scrape city info
!curl 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_California' > cal_city.dat
htmlfile = open("cal_city.dat", "r")
htmltxt = htmlfile.read()
#soup = BeautifulSoup(htmltxt, 'lxml')
soup = BeautifulSoup(htmltxt, 'html.parser')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  366k  100  366k    0     0   486k      0 --:--:-- --:--:-- --:--:--  485k


In [265]:
# Get target table and find out all tr tags
tbl = soup.find('table', attrs={'class':'sortable'})
tbl_html = "<html><body>" + str(tbl) + "</body></html>"
soup_tbl = BeautifulSoup(tbl_html, 'lxml')
trs = soup_tbl.find_all('tr')

In [275]:
# Collect scraped city data and put into a DataFrame
import math

name_list = []
population_list = []
size_list = []
radius_list = []
min_popu = 100000 # Threshould of city population

for n in range(2, len(trs)):
    row_data = trs[n].text.split('\n\n')
    #print(row_data)
    name = row_data[0].replace('\n','')
    if name.__contains__('Ontar') or name.__contains__('Stoc'):
        continue
    popu = row_data[-4].replace(',', '')
    size = row_data[-2].replace(',', '')
    if popu.__contains__('[') or size.__contains__('['):
        continue
    radius = int(np.sqrt(float(size)/math.pi) * 1000)

    if int(popu) >= min_popu:
        name_list.append(name)
        population_list.append(popu)
        size_list.append(size)
        radius_list.append(radius)
    

### b. City Geo Location Info

In [267]:
# Get geo location info for all cities in our dataframe
import time

lat_list = []
long_list = []

for city_name in df_cal_cities['Name']:
    address = city_name + ', CA'
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lat_list.append(latitude)
    long_list.append(longitude)
    print('The geograpical coordinate of {} are {}, {}.'.format(city_name, latitude, longitude))
    time.sleep(5)

/Users/MichaelChen/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Anaheim are 33.8347516, -117.911732.
The geograpical coordinate of Antioch are 38.0049214, -121.805789.
The geograpical coordinate of Bakersfield are 35.3738712, -119.0194639.
The geograpical coordinate of Berkeley are 53.5417337, -113.520773650607.
The geograpical coordinate of Burbank are 34.1816482, -118.3258554.
The geograpical coordinate of Carlsbad are 33.1580933, -117.3505966.
The geograpical coordinate of Chula Vista are 32.6400541, -117.0841955.
The geograpical coordinate of Concord are 45.452798, -62.7905136.
The geograpical coordinate of Corona are 49.198057, -124.087538.
The geograpical coordinate of Costa Mesa are 33.6633386, -117.903317.
The geograpical coordinate of Daly City are 37.7057671, -122.4619205.
The geograpical coordinate of Downey are 33.9400143, -118.1325688.
The geograpical coordinate of El Monte are 44.2434162, 10.8186516.
The geograpical coordinate of Elk Grove are 38.4087993, -121.3716178.
The geograpical coordinate of Escond

### c. Create DataFrame Of California Cities With Population > 100,000

In [278]:
df_cal_cities = pd.DataFrame(np.column_stack([name_list, population_list, size_list, lat_list, long_list, radius_list]), 
                               columns=['Name', 'Population', 'Size', 'Latitude', 'Longitude', 'Radius'])
df_cal_cities = df_cal_cities.drop([35,56])
df_cal_cities

,Name,Population,Size,Latitude,Longitude,Radius
0,Anaheim,336265,129.1,33.8347516,-117.911732,6410
1,Antioch,102372,73.4,38.0049214,-121.805789,4833
2,Bakersfield,347483,368.2,35.3738712,-119.0194639,10825
3,Berkeley,112580,27.1,53.5417337,-113.520773650607,2937
4,Burbank,103340,44.9,34.1816482,-118.3258554,3780
5,Carlsbad,105328,97.7,33.1580933,-117.3505966,5576
6,Chula Vista,243916,128.5,32.6400541,-117.0841955,6395
7,Concord,122067,79.1,45.452798,-62.7905136,5017
8,Corona,152374,100.6,49.198057,-124.087538,5658
9,Costa Mesa,109960,40.5,33.6633386,-117.903317,3590


### d. Display Target Cities On Map

In [279]:
    address = 'Irvine, CA'
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(city_name, latitude, longitude))


/Users/MichaelChen/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of West Covina are 33.6856969, -117.8259819.


In [280]:
# create map of Toronto using latitude and longitude values
map_ca = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, r in zip(df_cal_cities['Latitude'], df_cal_cities['Longitude'], df_cal_cities['Name'], df_cal_cities['Radius']):
    label = '{}, {}'.format(df_cal_cities, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius= int(float(r)/1000.0),
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ca)  
    
map_ca


## III. Prepare For Using FourSquare API

### Setup API Credentials

In [281]:
# FourSquare client info for querying its API
CLIENT_ID = 'Z4RVBOLDX2XR5RGLQBH2BGADEVLTCC5UHRISORQ3GE0K33EV' # your Foursquare ID
CLIENT_SECRET = 'Y4LIURUKTD5E5A3DIHGEVKJBSW1RP5WZLDNLAOQBG4EJALZF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z4RVBOLDX2XR5RGLQBH2BGADEVLTCC5UHRISORQ3GE0K33EV
CLIENT_SECRET:Y4LIURUKTD5E5A3DIHGEVKJBSW1RP5WZLDNLAOQBG4EJALZF


### Convenience Functions

In [282]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [283]:
# Function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius):
    n = 0
    venues_list=[]
    for name, lat, lng, r in zip(names, latitudes, longitudes, radius):
        
        n = n + 1
        print(str(n) + ": Getting venues for " + name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            r, 
            LIMIT)
        print("URL = " + url)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## IV. Fetching Venue Info For Each City

In [205]:
# Run above function for each city in our target cities
LIMIT = 100

cal_city_venues = getNearbyVenues(names=df_cal_cities['Name'], latitudes=df_cal_cities['Latitude'], longitudes=df_cal_cities['Longitude'], radius=df_cal_cities['Radius'])

1: Getting venues for Anaheim
URL = https://api.foursquare.com/v2/venues/explore?&client_id=Z4RVBOLDX2XR5RGLQBH2BGADEVLTCC5UHRISORQ3GE0K33EV&client_secret=Y4LIURUKTD5E5A3DIHGEVKJBSW1RP5WZLDNLAOQBG4EJALZF&v=20180605&ll=33.8347516,-117.911732&radius=6410&limit=100
2: Getting venues for Antioch
URL = https://api.foursquare.com/v2/venues/explore?&client_id=Z4RVBOLDX2XR5RGLQBH2BGADEVLTCC5UHRISORQ3GE0K33EV&client_secret=Y4LIURUKTD5E5A3DIHGEVKJBSW1RP5WZLDNLAOQBG4EJALZF&v=20180605&ll=38.0049214,-121.805789&radius=4833&limit=100
3: Getting venues for Bakersfield
URL = https://api.foursquare.com/v2/venues/explore?&client_id=Z4RVBOLDX2XR5RGLQBH2BGADEVLTCC5UHRISORQ3GE0K33EV&client_secret=Y4LIURUKTD5E5A3DIHGEVKJBSW1RP5WZLDNLAOQBG4EJALZF&v=20180605&ll=35.3738712,-119.0194639&radius=10825&limit=100
4: Getting venues for Berkeley
URL = https://api.foursquare.com/v2/venues/explore?&client_id=Z4RVBOLDX2XR5RGLQBH2BGADEVLTCC5UHRISORQ3GE0K33EV&client_secret=Y4LIURUKTD5E5A3DIHGEVKJBSW1RP5WZLDNLAOQBG4EJALZF&

In [206]:
print(cal_city_venues.shape)
cal_city_venues.head(30)

(5416, 7)


,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Anaheim,33.8347516,-117.911732,Anaheim Packing House,33.831546,-117.911946,Historic Site
1,Anaheim,33.8347516,-117.911732,Hans' Homemade Ice Cream,33.831560,-117.911842,Ice Cream Shop
2,Anaheim,33.8347516,-117.911732,The Blind Rabbit,33.831551,-117.911909,Speakeasy
3,Anaheim,33.8347516,-117.911732,Umami Burger,33.832693,-117.912717,Burger Joint
4,Anaheim,33.8347516,-117.911732,Unsung Brewing Company,33.830957,-117.912098,Brewery
5,Anaheim,33.8347516,-117.911732,The Rinks Anaheim Ice,33.833846,-117.917367,Skating Rink
6,Anaheim,33.8347516,-117.911732,Healthy Junk,33.833971,-117.915513,Vegetarian / Vegan Restaurant
7,Anaheim,33.8347516,-117.911732,Orange Roll & Sushi,33.832435,-117.913140,Sushi Restaurant
8,Anaheim,33.8347516,-117.911732,PokiNometry,33.832941,-117.918900,Poke Place
9,Anaheim,33.8347516,-117.911732,Tony's Deli,33.828340,-117.911728,Deli / Bodega


In [251]:
# Display number of venues in each city
cal_city_venues_grouped = cal_city_venues.groupby('Name').count()
cal_city_venues_grouped

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Name,,,,,,
Anaheim,100,100,100,100,100,100
Antioch,100,100,100,100,100,100
Bakersfield,100,100,100,100,100,100
Berkeley,100,100,100,100,100,100
Burbank,100,100,100,100,100,100
Carlsbad,100,100,100,100,100,100
Chula Vista,100,100,100,100,100,100
Concord,3,3,3,3,3,3
Corona,47,47,47,47,47,47


## V. Venue Category Analysis

### a. Display Number Of Unique Categories

In [252]:
# See unique category of the venues
print('There are {} uniques categories in California cities.'.format(len(cal_city_venues['Venue Category'].unique())))

There are 325 uniques categories in California cities.


### b. One-Hot Encoding Categories

In [211]:
# Analyse venue categories

# one hot encoding
cal_city_venues_onehot = pd.get_dummies(cal_city_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
cal_city_venues_onehot['Name'] = cal_city_venues['Name'] 

# move city column to the first column
fixed_columns = [cal_city_venues_onehot.columns[-1]] + list(cal_city_venues_onehot.columns[:-1])
cal_city_venues_onehot = cal_city_venues_onehot[fixed_columns]

cal_city_venues_onehot.head(10)

,Name,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Emilia Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Observatory,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speak

In [146]:
cal_city_venues_onehot.shape

(5416, 326)

In [212]:
# Calculate mean and freq in each city
cal_city_grouped = cal_city_venues_onehot.groupby('Name').mean().reset_index()
cal_city_grouped

,Name,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Emilia Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Observatory,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speak

In [213]:
cal_city_grouped.shape

(64, 326)

In [214]:
# Print top 5 most common in each city
num_top_venues = 5

for city in cal_city_grouped['Name']:
    print("----"+city+"----")
    temp = cal_city_grouped[cal_city_grouped['Name'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anaheim----
                          venue  freq
0  Theme Park Ride / Attraction  0.32
1                    Theme Park  0.11
2                   Art Gallery  0.03
3                Ice Cream Shop  0.03
4                  Burger Joint  0.03


----Antioch----
                  venue  freq
0  Fast Food Restaurant  0.10
1           Coffee Shop  0.07
2        Sandwich Place  0.06
3              Pharmacy  0.05
4           Pizza Place  0.04


----Bakersfield----
                venue  freq
0  Mexican Restaurant  0.13
1       Grocery Store  0.07
2         Pizza Place  0.05
3          Steakhouse  0.04
4        Burger Joint  0.04


----Berkeley----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.07
2                Café  0.06
3         Pizza Place  0.04
4          Restaurant  0.04


----Burbank----
                 venue  freq
0       Sandwich Place  0.07
1   Mexican Restaurant  0.05
2  American Restaurant  0.05
3          Pizza Place  0.04
4                D

In [215]:
# Function to sort venue in dec order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [216]:
# Display top 10 venues in each city
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cal_city_venues_sorted = pd.DataFrame(columns=columns)
cal_city_venues_sorted['Name'] = cal_city_grouped['Name']

for ind in np.arange(cal_city_grouped.shape[0]):
    cal_city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cal_city_grouped.iloc[ind, :], num_top_venues)

cal_city_venues_sorted

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anaheim,Theme Park Ride / Attraction,Theme Park,Gift Shop,Ice Cream Shop,Burger Joint,Art Gallery,Cajun / Creole Restaurant,Hotel,Steakhouse,American Restaurant
1,Antioch,Fast Food Restaurant,Coffee Shop,Sandwich Place,Pharmacy,Burger Joint,Pizza Place,Grocery Store,Mexican Restaurant,Supermarket,Movie Theater
2,Bakersfield,Mexican Restaurant,Grocery Store,Pizza Place,Steakhouse,Burger Joint,Bakery,Coffee Shop,Chinese Restaurant,American Restaurant,Hotel
3,Berkeley,Italian Restaurant,Coffee Shop,Café,Pizza Place,Restaurant,Bar,Nightclub,Indian Restaurant,Diner,Breakfast Spot
4,Burbank,Sandwich Place,Mexican Restaurant,American Restaurant,Diner,Pizza Place,Burger Joint,Japanese Restaurant,Donut Shop,Coffee Shop,Fast Food Restaurant
5,Carlsbad,Beach,Café,Coffee Shop,Mexican Restaurant,Seafood Restaurant,Breakfast Spot,Pizza Place,American Restaurant,Grocery Store,Juice Bar
6,Chula Vista,Mexican Restaurant,Coffee Shop,Taco Place,Burger Joint,Hotel,Clothing Store,Gym / Fitness Center,Seafood Restaurant,Italian Restaurant,American Restaurant
7,Concord,Recreation Center,Construction & Landscaping,Antique Shop,Zoo Exhibit,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
8,Corona,Coffee Shop,Sushi Restaurant,Fast Food Restaurant,Pet Store,BBQ Joint,Grocery Store,Breakfast Spot,Shopping Mall,Tea Room,Supermarket
9,Costa Mesa,Coffee Shop,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place,Grocery Store,Mexican Restaurant,Burger Joint,Shopping Mall,Bakery


## VI. City Clustering Analysis Via Venues Data

In [253]:
# Run k-mean to cluster target cities into 20 clusters
kclusters = 20

cal_city_grouped_clustering = cal_city_grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cal_city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([10, 19,  0, 13,  3, 14, 11,  4,  5,  3,  3, 11,  6, 11,  0,  2, 19,
       16,  0,  3, 16,  3, 17,  0, 19,  3, 15, 14,  3,  0, 11, 11, 19, 12,
       14, 18,  0, 11,  3,  0, 11, 16,  7,  1, 15, 19,  0, 13, 13,  3,  0,
        3, 11, 13, 11,  3, 11, 11,  9, 11,  8, 19, 11, 11], dtype=int32)

### a. Target Cities With Top10 Venues and Cluster Labels

In [248]:
# Put cluster label in the original dataframe
cal_city_merged = df_cal_cities

# add clustering labels
cal_city_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cal_city_merged = cal_city_merged.join(cal_city_venues_sorted.set_index('Name'), on='Name')

cal_city_merged # check the last columns!

,Name,Population,Size,Latitude,Longitude,Radius,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anaheim,336265,129.1,33.8347516,-117.911732,6410,10,Theme Park Ride / Attraction,Theme Park,Gift Shop,Ice Cream Shop,Burger Joint,Art Gallery,Cajun / Creole Restaurant,Hotel,Steakhouse,American Restaurant
1,Antioch,102372,73.4,38.0049214,-121.805789,4833,19,Fast Food Restaurant,Coffee Shop,Sandwich Place,Pharmacy,Burger Joint,Pizza Place,Grocery Store,Mexican Restaurant,Supermarket,Movie Theater
2,Bakersfield,347483,368.2,35.3738712,-119.0194639,10825,0,Mexican Restaurant,Grocery Store,Pizza Place,Steakhouse,Burger Joint,Bakery,Coffee Shop,Chinese Restaurant,American Restaurant,Hotel
3,Berkeley,112580,27.1,53.5417337,-113.520773650607,2937,13,Italian Restaurant,Coffee Shop,Café,Pizza Place,Restaurant,Bar,Nightclub,Indian Restaurant,Diner,Breakfast Spot
4,Burbank,103340,44.9,34.1816482,-118.3258554,3780,3,Sandwich Place,Mexican Restaurant,American Restaurant,Diner,Pizza Place,Burger Joint,Japanese Restaurant,Donut Shop,Coffee Shop,Fast Food Restaurant
5,Carlsbad,105328,97.7,33.1580933,-117.3505966,5576,14,Beach,Café,Coffee Shop,Mexican Restaurant,Seafood Restaurant,Breakfast Spot,Pizza Place,American Restaurant,Grocery Store,Juice Bar
6,Chula Vista,243916,128.5,32.6400541,-117.0841955,6395,11,Mexican Restaurant,Coffee Shop,Taco Place,Burger Joint,Hotel,Clothing Store,Gym / Fitness Center,Seafood Restaurant,Italian Restaurant,American Restaurant
7,Concord,122067,79.1,45.452798,-62.7905136,5017,4,Recreation Center,Construction & Landscaping,Antique Shop,Zoo Exhibit,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
8,Corona,152374,100.6,49.198057,-124.087538,5658,5,Coffee Shop,Sushi Restaurant,Fast Food Restaurant,Pet Store,BBQ Joint,Grocery Store,Breakfast Spot,Shopping Mall,Tea Room,Supermarket
9,Costa Mesa,109960,40.5,33.6633386,-117.903317,3590,3,Coffee Shop,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place,Grocery Store,Mexican Restaurant,Burger Joint,Shopping Mall,Bakery


### b. Clustered Target Cities Displayed On Map

Note: Currently the map is still not quite perfect, some cities has not displayed, and some cluster labels not seen on map at all (though they exist in the DF).

In [254]:
# Visualize the clusters on our map
map_clusters = folium.Map(location=[33.6857, -117.8259], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#rainbow = ['#a20404','#f47b67','#ff3478','#ff9ecb','#ca6e7e','#1500cc','#9c84ef','#49a4aa','#b9f645','#cba081']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cal_city_merged['Latitude'], cal_city_merged['Longitude'], cal_city_merged['Name'], cal_city_merged['Cluster Labels']):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### c. The Irvine Cluster (#3) - The California Cities That Feels Like Irvine (Well, venue-wise?)

In [262]:
irvine_flag = cal_city_merged['Cluster Labels'] == 3

In [257]:
irvine_like_cities = cal_city_merged[irvine_flag]

### d. There Are 10 Other Cities In The Irvine Cluster

In [258]:
irvine_like_cities

,Name,Population,Size,Latitude,Longitude,Radius,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Burbank,103340,44.9,34.1816482,-118.3258554,3780,3,Sandwich Place,Mexican Restaurant,American Restaurant,Diner,Pizza Place,Burger Joint,Japanese Restaurant,Donut Shop,Coffee Shop,Fast Food Restaurant
9,Costa Mesa,109960,40.5,33.6633386,-117.903317,3590,3,Coffee Shop,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place,Grocery Store,Mexican Restaurant,Burger Joint,Shopping Mall,Bakery
10,Daly City,101123,19.8,37.7057671,-122.4619205,2510,3,Pizza Place,Burger Joint,Coffee Shop,Café,Bakery,BBQ Joint,Chinese Restaurant,Japanese Restaurant,Filipino Restaurant,Grocery Store
19,Fullerton,135161,57.9,33.8708215,-117.9294165,4293,3,Coffee Shop,Sushi Restaurant,Burger Joint,Italian Restaurant,Mexican Restaurant,Breakfast Spot,Ice Cream Shop,Café,Sports Bar,Salon / Barbershop
21,Glendale,191719,78.9,51.034096,-114.1522951,5011,3,Coffee Shop,Japanese Restaurant,Breakfast Spot,Pub,Ice Cream Shop,Restaurant,Fast Food Restaurant,Pizza Place,Diner,Bakery
25,Irvine,212375,171.2,33.6856969,-117.8259819,7382,3,Burger Joint,American Restaurant,Fast Food Restaurant,Park,Grocery Store,Pizza Place,Ice Cream Shop,Coffee Shop,Korean Restaurant,Sandwich Place
28,Los Angeles,3792621,1213.8,34.0536834,-118.2427669,19656,3,Coffee Shop,Korean Restaurant,Ice Cream Shop,Theater,Sandwich Place,Wine Shop,Park,Italian Restaurant,Mexican Restaurant,Trail
39,Pasadena,137122,59.5,34.1476452,-118.1444779,4351,3,Coffee Shop,American Restaurant,Cosmetics Shop,Pizza Place,Gym / Fitness Center,Italian Restaurant,Bakery,Mediterranean Restaurant,Steakhouse,Pub
50,San Jose,945942,457.2,37.3361905,-121.8905833,12063,3,Sandwich Place,Mexican Restaurant,Pizza Place,Coffee Shop,Vietnamese Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Bar,Ice Cream Shop,Grocery Store
52,Santa Clara,116468,47.7,37.3541132,-121.9551744,3896,3,Korean Restaurant,Pizza Place,Japanese Restaurant,Sandwich Place,Indian Restaurant,Grocery Store,Thai Restaurant,Coffee Shop,Burger Joint,Gym / Fitness Center
